In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

# Load the dataset
df = pd.read_csv('/Users/aryankargwal/adverscredit/models/data/credit.csv')

# Split the dataset into features and target
X = df.drop('Class', axis=1).values
y = df['Class'].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()

# PGD attack function
def pgd_attack(model, X, y, epsilon=0.01, alpha=0.001, num_iter=10):
    X_adv = X.clone().detach().requires_grad_(True)
    for _ in range(num_iter):
        model.zero_grad()
        outputs = model(X_adv)
        loss = criterion(outputs.squeeze(), y)
        loss.backward()
        with torch.no_grad():
            X_adv += alpha * X_adv.grad.sign()
            eta = torch.clamp(X_adv - X, min=-epsilon, max=epsilon)
            X_adv = torch.clamp(X + eta, min=0, max=1).detach_()
            X_adv.requires_grad_(True)
    return X_adv

# Generate adversarial samples
X_train_adv = pgd_attack(model, X_train, y_train)
X_test_adv = pgd_attack(model, X_test, y_test)

# Combine adversarial and original samples
def scatter_adversarial_samples(X, X_adv, y):
    X_combined = X.clone().detach().cpu().numpy()
    y_combined = y.clone().detach().cpu().numpy()
    X_adv_np = X_adv.clone().detach().cpu().numpy()
    
    num_samples = len(y)
    num_adv_samples = len(y) // 10  # Adding adversarial samples equal to 10% of original samples
    indices = np.random.choice(num_samples, num_adv_samples, replace=False)
    
    X_combined[indices] = X_adv_np[indices]
    
    return X_combined, y_combined

X_train_combined, y_train_combined = scatter_adversarial_samples(X_train, X_train_adv, y_train)
X_test_combined, y_test_combined = scatter_adversarial_samples(X_test, X_test_adv, y_test)

# Save the new datasets with adversarial samples
train_combined_df = pd.DataFrame(X_train_combined, columns=df.columns[:-1])
train_combined_df['Class'] = y_train_combined
train_combined_df.to_csv('pgd_train.csv', index=False)

test_combined_df = pd.DataFrame(X_test_combined, columns=df.columns[:-1])
test_combined_df['Class'] = y_test_combined
test_combined_df.to_csv('pgd_test.csv', index=False)

print("Adversarial samples generated and saved to pgd_train.csv and pgd_test.csv")


Adversarial samples generated and saved to pgd_train.csv and pgd_test.csv
